### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [8]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [20]:
num_iterations = 1
empty_list = []
for i in range(32, 42):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.62


######### Lightcurve features #########

amplitude = 0.2369845630043828
cusum = 0.1063749067256072
eta = 2.5492043627985868
hl_amp_ratio = 0.5651054804105654
kurtosis = 5.337904248776386
n_points = 1847
period = 0.6060623328844292
period_SNR = 247.65578602781895
period_log10FAP = -201.28017952899867
period_uncertainty = 0.00015901584080973974
phase_cusum = 0.22431725152252568
phase_eta = 0.9015753091865975
phi21 = -2.2977993557264997
phi31 = -1.0609156591021476
quartile31 = 0.33931536624393566
r21 = 0.42785883527218294
r31 = 0.26859340269793963
shapiro_w = 0.8913807272911072
skewness = 0.6314187758706398
slope_per10 = -0.06000154715053636
slope_per90 = 0.05218651231803488
stetson_k = 0.8128080628831114
weighted_mean = 12.175689438371041
weighted_std = 0.1959164007992105
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/upsilon/extract_features/extract_features.py:204: RuntimeWarning: divide by zero encountered in log10
  np.log10(pLS.getSignificance(fx



######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.97


######### Lightcurve features #########

amplitude = 0.24847818656683338
cusum = 0.3891916662175626
eta = 1.926034978998021
hl_amp_ratio = 1.140668226033972
kurtosis = -0.8147868025317684
n_points = 2097
period = 0.3332001232378649
period_SNR = 275.3944086047456
period_log10FAP = -inf
period_uncertainty = 4.773640832045345e-05
phase_cusum = 0.42789147465759386
phase_eta = 0.745222427249245
phi21 = 2.036357636115789
phi31 = 3.7867198101063835
quartile31 = 0.33999076153260255
r21 = 0.12272588063733185
r31 = 0.08437581736100085
shapiro_w = 0.9692638516426086
skewness = 0.27175595902008715
slope_per10 = -0.009579639532206828
slope_per90 = 0.008059693352264448
stetson_k = 0.890251701938397
weighted_mean = 11.155126926027988
weighted_std = 0.17081949835104232
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.98


######### Lightcurve features #########

amplitude = 0.46718792238191287
cusum = 0.3269117380337446
eta = 1.857026176602375
hl_amp_ratio = 0.5781562701482448
kurtosis = -0.1439445633853218
n_points = 2256
period = 0.45068259414893297
period_SNR = 257.1491886457448
period_log10FAP = -199.53378097099522
period_uncertainty = 8.77041926342359e-05
phase_cusum = 0.4436341442117096
phase_eta = 0.8742652616742058
phi21 = 0.5873466711420198
phi31 = -1.6401571505005563
quartile31 = 0.5998095149540905
r21 = 0.4757145676512582
r31 = 0.3292568586928446
shapiro_w = 0.98606938123703
skewness = -0.20940112449047718
slope_per10 = -0.004382952549215311
slope_per90 = 0.0043589660526580105
stetson_k = 0.6401967840360618
weighted_mean = 15.265658319741755
weighted_std = 0.5061134060723245
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.82


######### Lightcurve features #########

amplitude = 0.21800617682319945
cusum = 0.12781140164561136
eta = 1.4302460545881313
hl_amp_ratio = 0.9595645977647695
kurtosis = -1.3994370707599022
n_points = 1612
period = 0.35013826682239474
period_SNR = 228.344798188063
period_log10FAP = -300.84422668529834
period_uncertainty = 4.396071114420863e-05
phase_cusum = 0.3257212741148413
phase_eta = 0.206532628986435
phi21 = 1.291802810536906
phi31 = 3.594837046630942
quartile31 = 0.3269112384232091
r21 = 0.1047160040596394
r31 = 0.048929201624902455
shapiro_w = 0.929752767086029
skewness = 0.0014692034738598451
slope_per10 = -0.020843665625578087
slope_per90 = 0.023310891015992947
stetson_k = 0.8417648464776069
weighted_mean = 13.833126059929999
weighted_std = 0.15816132879700273


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
